In [1]:
import csv
from gensim import corpora, models, similarities
from collections import defaultdict
from six import iteritems
import re

In [5]:
stoplist = []
documents = []

with open("stopwords-en.csv", 'r') as csvfile:
    csvreader = csv.reader(csvfile)
    for row in csvreader:
        stoplist.append(row[0])

with open(filename, 'r') as csvfile:
    csvreader = csv.reader(csvfile)
    for row in csvreader:
        documents.append(row[1])

#Eliminate numbers so they don't gunk up the analysis-- but preserve them as a token so they are not invisibilized.
documents =	[re.sub('\b[0-9][0-9.,-]*\b', 'NUMBER-TOKEN', line) for line in documents]

NameError: name 'documents' is not defined

In [3]:
read_data(filename="ap.csv")
#Make the dictionary that Gensim will use
dictionary = corpora.Dictionary(line.lower().split() for line in documents)
stop_ids = [dictionary.token2id[stopword] for stopword in stoplist if stopword in dictionary.token2id]
once_ids = [tokenid for tokenid, docfreq in iteritems(dictionary.dfs) if docfreq == 1]
dictionary.filter_tokens(stop_ids + once_ids)
dictionary.compactify()

#Convert each document to a bag of words
class MyCorpus(object):
    def __iter__(self):
        for line in documents:
            yield dictionary.doc2bow(line.lower().split(' '))

corpus = MyCorpus()

tfidf = models.TfidfModel(corpus)
tfidf_corpus = tfidf[corpus]

model = models.LsiModel(tfidf_corpus, id2word=dictionary, num_topics=300)


NameError: name 'documents' is not defined

In [10]:
import random
topics = model.print_topics()
random.shuffle(topics)
topics[:10]

[(3,
  '-0.290*"police" + 0.284*"bush" + 0.244*"soviet" + 0.199*"dukakis" + 0.132*"gorbachev" + -0.091*"fire" + 0.091*"reagan" + 0.088*"trade" + 0.087*"budget" + -0.085*"killed"'),
 (17,
  '-0.253*"israeli" + -0.246*"israel" + 0.206*"german" + -0.155*"palestinian" + -0.145*"jewish" + -0.137*"arab" + -0.136*"plo" + -0.134*"palestinians" + -0.124*"occupied" + -0.122*"inches"'),
 (14,
  '0.184*"dukakis" + -0.181*"police" + 0.162*"company" + -0.160*"trade" + 0.139*"jackson" + -0.134*"budget" + -0.130*"bush" + -0.127*"bill" + -0.111*"drug" + 0.107*"south"'),
 (0,
  '0.113*"soviet" + 0.107*"percent" + 0.105*"bush" + 0.089*"police" + 0.088*"u.s." + 0.085*"will" + 0.080*"_" + 0.080*"million" + 0.078*"new" + 0.077*"government"'),
 (4,
  '0.409*"soviet" + -0.187*"dukakis" + 0.182*"gorbachev" + -0.154*"percent" + 0.134*"stock" + 0.132*"index" + -0.103*"bush" + -0.098*"dollar" + 0.091*"communist" + -0.083*"billion"'),
 (18,
  '0.228*"police" + 0.182*"bush" + 0.142*"oil" + -0.136*"percent" + -0.127

In [11]:
model = models.LdaModel(tfidf_corpus, id2word=dictionary, num_topics=300)


/usr/local/lib/python3.6/site-packages/gensim/models/ldamodel.py:1023: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


In [13]:
topics = model.print_topics()
random.shuffle(topics)

In [14]:
topics[:10]

[(268,
  '0.014*"survey" + 0.014*"filing" + 0.012*"average" + 0.010*"type" + 0.007*"massachusetts." + 0.006*"families." + 0.006*"counseling" + 0.006*"activated" + 0.006*"raided" + 0.005*"out."'),
 (155,
  '0.022*"rig" + 0.016*"presley" + 0.015*"10-minute" + 0.011*"elvis" + 0.011*"disc" + 0.010*"peoples" + 0.008*"presley\'s" + 0.008*"sleeping" + 0.008*"moviegoers" + 0.008*"explosion,"'),
 (39,
  '0.021*"mecham" + 0.013*"refugees" + 0.012*"asylum-seekers" + 0.010*"taipei." + 0.008*"refugee" + 0.007*"milstead" + 0.007*"counseling" + 0.006*"11:30" + 0.006*"up," + 0.006*"patients\'"'),
 (100,
  '0.023*"gun" + 0.021*"killers" + 0.013*"black-jewish" + 0.012*"law" + 0.009*"cheap" + 0.008*"maryland\'s" + 0.008*"handguns" + 0.007*"trendy" + 0.006*"maryland" + 0.006*"alliance"'),
 (150,
  '0.013*"illegally" + 0.013*"joints" + 0.011*"testimony" + 0.008*"bodies," + 0.007*"quickly" + 0.007*"watergate" + 0.006*"evidence" + 0.005*"client" + 0.005*"haldeman" + 0.005*"cries"'),
 (130,
  '0.008*"well-to-

In [8]:
import pandas as pd
from textblob import TextBlob
from sklearn.feature_extraction.text import TfidfVectorizer

In [14]:
df = pd.read_csv("state-of-the-union.csv")
df.columns = ['Year','Speech']

In [16]:
df = df[(df.Year > 1899) & (df.Year < 2000)]
df.reset_index(inplace=True)

In [18]:
def tokenize(s):
    blob = TextBlob(s.lower())
    words = [token for token in blob.words if len(token)>2]
    return words

vectorizer = TfidfVectorizer(stop_words='english', tokenizer=tokenize)

matrix = vectorizer.fit_transform(df.Speech)

tfidf = pd.DataFrame(matrix.toarray(), columns=vectorizer.get_feature_names())

decade_rows = []
for i in range(1900, 2000, 10):
    decade_df = tfidf[(df.Year > i) & (df.Year < i + 10)]
    decade_sum = decade_df.sum()
    df_length = len(decade_df)
    dec_vec = decade_sum / df_length
    decade_rows.append(dec_vec)
    
dec_tfidf = pd.DataFrame(decade_rows)

def print_sorted_vector(v):
    sorted_list = sorted(v.items(), key=lambda x: (x[1],x[0]), reverse=True) 
    sorted_list = sorted_list[:20]
    print('\n'.join([str(x) for x in sorted_list]))

In [21]:
for decade in range(len(dec_tfidf)):
    print("Decade: ",decade)
    print_sorted_vector(dec_tfidf.iloc[decade])
    


Decade:  0
('government', 0.1382503617705634)
('law', 0.11214868849049951)
('states', 0.10493024322618973)
('great', 0.10001961369510598)
('congress', 0.08643588577999645)
('people', 0.0833412938587434)
('public', 0.08220850400633266)
('country', 0.081249008099739)
('united', 0.08084147096231559)
('men', 0.08030697027292416)
('work', 0.0799965563504776)
('man', 0.07711764895698298)
('navy', 0.07470491372634279)
('corporations', 0.07445474083645891)
('business', 0.07330600904473168)
('officers', 0.06978180096697983)
('commerce', 0.068411113430638)
('service', 0.06472747201514999)
('department', 0.06469793655406657)
('islands', 0.06428332771244921)
Decade:  1
('government', 0.11157823462813592)
('shall', 0.08862784515269287)
('great', 0.08181069885482202)
('congress', 0.08150409836345998)
('country', 0.07905992904838366)
('states', 0.07730897844553355)
('war', 0.07208245824536633)
('men', 0.06681689735962684)
('united', 0.06368153298067744)
('people', 0.06231947653699088)
('necessary', 0